In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **IMPORTS**

In [2]:
from pathlib import Path

root_to_data = Path('/content/drive/MyDrive/gagarin_dataset')

In [45]:
%%time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statistics
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

!pip install --upgrade transformers
!pip install transformers[torch]
!pip install accelerate -U

import transformers

print(f'transformer varsion: {transformers.__version__}')

plt.style.use('ggplot')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# **DATA**

In [4]:
sentiment_labels = pd.read_csv(f'{root_to_data}/sentiment.csv')
sentiment_labels.drop('Unnamed: 0',axis = 1,inplace = True)
print(sentiment_labels.shape, '\n', sentiment_labels.head(3))

(10931, 4) 
    MessageID   ChannelID  issuerid  SentimentScore
0     241407  1203560567       153               2
1      33684  1136626166       230               4
2      10090  1063908560       118               4


In [5]:
sentiment_text = pd.read_pickle(f'{root_to_data}/sentiment_texts.pickle')
print(sentiment_text.shape, '\n', sentiment_text.head(3))

(9289, 8) 
    MessageID   ChannelID  issuerid  SentimentScore           DateAdded  \
0     241407  1203560567       153               2 2023-05-12 19:03:20   
1      33684  1136626166       230               4 2023-02-03 20:56:29   
2      10090  1063908560       118               4 2023-06-02 19:18:37   

           DatePosted                                        MessageText  \
0 2023-05-12 19:02:42  ⚠️🇷🇺#SELG #дивиденд  сд Селигдар: дивиденды 20...   
1 2023-02-03 16:46:34  Ozon продолжает развивать специализированные ф...   
2 2023-06-02 18:50:00  ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...   

   IsForward  
0      False  
1      False  
2      False  


In [6]:
# Сделаем слияние sentiment_labels и sentiment_text
summarization_df = sentiment_labels.merge(sentiment_text, how='right', on='MessageID')

summarization_df.drop(['ChannelID_y','issuerid_y','SentimentScore_y', 'MessageID', 'ChannelID_x', 'issuerid_x', 'SentimentScore_x', 'DateAdded', 'DatePosted', 'IsForward'], axis=1, inplace=True)

summarization_df = summarization_df.drop_duplicates(subset=['MessageText'], keep='first')

print(summarization_df.shape, '\n', summarization_df.head(5))

(7188, 1) 
                                           MessageText
0   ⚠️🇷🇺#SELG #дивиденд  сд Селигдар: дивиденды 20...
1   Ozon продолжает развивать специализированные ф...
2   ​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ ...
6   ​​Windfall Tax — налог на сверхприбыль. Какие ...
22  У нас было 2 пакета префов Сургутнефтегаза, 75...


**Для наилучшего результата суммаризации стоит убрать все смайлики** 🔥

In [7]:
sentence = summarization_df['MessageText'][2]
print(sentence)
sentence_clean = re.sub('[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]+', '', sentence)
print(sentence_clean)

​Фокусы продолжаются🔥Акции и инвестиции  📈ВТБ +5.1% ВТБ сегодня показывает чудеса на виражах! В моменте терял более 5.5% на новостях о цене размещения допэмиссии, которая оказалась почти на 15% ниже котировок закрытия четверга(цена размещения 0,018225 рублей). Тем не менее, после падения котировки начали уверенно отскакивать и акции вышли в лидеры роста😳  📉Мечел ап -0.5% Еще один фокусник дня — это Мечел. В моменте терял более 5.5%, СД рекомендовал не выплачивать дивиденды по итогам 2022 года. Компания получила убыток по итогам 2022 года в размере 33 258 899 рублей 89 копеек. Несмотря на это, акции к концу дня вышли в плюс🤷‍♂️  📈Мосбиржа +4.4% Акционеры обрадовались утверждённому дивиденду(4.84 руб.). Кроме того, компания подвела итоги торгов в мае 2023 года, общий объем торгов на рынках Московской биржи вырос на 20,5%💪  📈HMSG +31.3% 📈MDMG +6.5% Идея нормализации ситуации в расписках продолжает поддерживать котировки🤔   📈РусГидро +3.7% СД рекомендовал 0,05025 рублей дивидендов. Не густ

In [8]:
list_messages = summarization_df['MessageText'].tolist()
clean_messages_df = pd.DataFrame(columns=['CleanSentences'])

for sent in list_messages:
  sent_clean = re.sub('[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]+', '', sent)
  clean_messages_df = pd.concat([clean_messages_df, pd.DataFrame({'CleanSentences': [sent_clean]})], ignore_index=True)

clean_messages_df.head(3)

,CleanSentences
0,#SELG #дивиденд сд Селигдар: дивиденды 2022г ...
1,Ozon продолжает развивать специализированные ф...
2,​Фокусы продолжаютсяАкции и инвестиции ВТБ +5...


In [9]:
clean_messages_df.shape

(7188, 1)

# **Для задачи суммаризации будем использовать sberbank-ai/rugpt3large_based_on_gpt2**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Загрузка предобученной модели T5 и токенизатора
model_name = 't5-base' # Используйте 't5-base' для английского языка
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Функция для суммаризации текста
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0])
    return summary

summary = summarize(sentence)
print(summary)

# def tokenize_function(examples):
#     inputs = tokenizer(examples["documents"], truncation=True, padding="max_length", max_length=1024)
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples["summaries"], truncation=True, padding="max_length", max_length=128)
#     inputs["labels"] = labels["input_ids"]
#     return inputs
# tokenized_dataset = your_dataset.map(tokenize_function, batched=True)

# model = BartForConditionalGeneration.from_pretrained('sberbank-ai/rugpt3large_based_on_gpt2')

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     push_to_hub=False,
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["test"],
#     tokenizer=tokenizer,
# )

# trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<pad> <unk> окус<unk> <unk> родол<unk> а<unk> тс<unk> ак<unk> ии инвести<unk> ии вт<unk> 5 1 вт<unk> се<unk> одн<unk> ока<unk> алас<unk> 15 ни<unk> е котировок <unk> акр<unk> ти<unk> <unk> етвер<unk> а <unk> ена ра<unk> ме<unk> ени<unk> 0 018225 ру<unk> ле<unk> менее <unk> адени<unk> коти</s>


In [ ]:
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0])
    return summary

summary = summarize(sentence)
print(summary)

AttributeError: 'WordPunctTokenizer' object has no attribute 'encode'

# **Подбираем предобученные модели с [*hugging face*](https://huggingface.co/) на примере из одного текста**

In [48]:
example = clean_messages_df['CleanSentences'][3]
example

'\u200b\u200bWindfall Tax — налог на сверхприбыль. Какие компании заплатят Минфину.  Многие понимают, что наш бюджет испытывает трудности и его необходимо наполнять всевозможными способами. Минфин использует различные финансовые рычаги: выход на долговой рынок — ОФЗ, продажа золота и юаней из ФНБ, но одним из главных является налоговый.   Как вы могли заметить, повышение НДПИ налог на добычу полезных ископаемых для различных отраслей в этом году уже состоялось. Причём некоторые заплатили разовый уже в прошлом и будут платить в этом: Газпром, Алроса. Это не считая того, что повысили процент на акцизы, штрафы, пошлины, тарифы и сборы.   Windfall Tax — совсем другая история.  Законопроект по налогу пока ещё не опубликован, но у заинтересованных лиц он уже есть на руках, его обсуждают различные издания и комментируют известные личности в финансовых кругах.   Налоговой базой для Windfall Tax будет превышение средней арифметической величины прибыли за 2021 год и за 2022 год над прибылью за 2

In [46]:
%%time
from transformers import T5ForConditionalGeneration, T5TokenizerFast, Trainer, TrainingArguments

# Зададим название выбронной модели из хаба
MODEL_NAME = 'UrukHan/t5-russian-summarization'

# Загрузка модели и токенизатора
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

CPU times: user 1.49 s, sys: 1.3 s, total: 2.8 s
Wall time: 11.1 s


In [49]:
inputs = tokenizer(example, return_tensors="pt", padding=True, truncation=True, max_length=1024)

# Генерация резюме
summary_ids = model.generate(**inputs)

# Декодирование выходных данных
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

«Сбербанк и ВТБ не могут договориться»: Минфин РФ о повышении НДПИ


# **Подготавливаем данные для файнтюнинга предобученной модели**

In [40]:
clean_messages_df['CleanSentences'][19]

'\u200b\u200bГолубые фишки с потенциалом +50%  Рынки акций радуют в этом году: с января в хорошем плюсе все эшелоны российских ценных бумаг, а также иностранные бумаги, доступные россиянам. Пройдемся по самым ликвидным акциям с высоким не реализованным потенциалом.  Голубые фишки РФ  Даже сейчас, после нескольких месяцев роста российский рынок остается недооцененным в рублях и может похвастаться высокой ожидаемой доходностью даже среди акций тяжеловесов.  Отберем бумаги, которые входят в состав Индекса МосБиржи голубых фишек топ-15 самых крупных и ликвидных, имеют рекомендацию не ниже «Покупать» и годовой таргет от аналитиков с потенциалом роста +50% и выше.  • Яндекс #YNDX Покупать. Цель на год: 3500 руб. / +82% • Татнефть #TATN Покупать. Цель на год: 610 руб. / +56% • Полюс #PLZL Покупать. Цель на год: 15000 руб. / +55%  Яндекс и Полюс — безусловные лидеры своих отраслей. Татнефть занимает удобную нишу регионального лидера. Все три ведут прозрачную финансовую политику, имеют драйверы

Пусть датасет для обучения будет состоять из 20 элементов

In [17]:
clean_sentence = clean_messages_df['CleanSentences'].iloc[0:20]
DataSet = pd.DataFrame(clean_sentence, columns=['CleanSentences'])
DataSet.head(3)

,CleanSentences
0,#SELG #дивиденд сд Селигдар: дивиденды 2022г ...
1,Ozon продолжает развивать специализированные ф...
2,​Фокусы продолжаютсяАкции и инвестиции ВТБ +5...


Я не смог найти датасеты в открытом доступе, которые подошли бы для этой задачи и для таких данных, поэтому сделал датасет вручную, пусть и не очень большой

In [41]:
DataSet.loc[0, 'summarize'] = "Селигдар не выплачивает дивиденды за 2022 год."
DataSet.loc[1, 'summarize'] = "Ozon запускает новый финтех-продукт «Деньги на закупки», позволяющий продавцам на маркетплейсе получать финансирование с отсрочкой платежа. Ozon оплачивает закупку товара, а продавцы вносят плату по мере появления средств на балансе в срок от 30 до 90 дней. Этот сервис упрощает процесс получения кредита, исключая необходимость собирать документы и искать поручителя."
DataSet.loc[2, 'summarize'] = "ВТБ показывает рост акций на 5.1% после падения из-за новостей о цене размещения допэмиссии, которая оказалась ниже ожиданий. Несмотря на это, акции восстановились и вышли в лидеры роста. Мечел, несмотря на падение более 5.5%, выходит в плюс после рекомендации СД не выплачивать дивиденды за 2022 год из-за убытка. Мосбиржа показывает рост на 4.4% после утверждения дивиденда. РусГидро и Совкомфлот также показывают рост, а Ростелеком и Лукойл отмечаются за рост выручки и падение после дивидендной отсечки соответственно."
DataSet.loc[3, 'summarize'] = "В России введена новая форма налога - Windfall Tax (налог на сверхприбыль), который будет взиматься с компаний, превысивших среднюю арифметическую прибыль за 2021 и 2022 годы по сравнению с 2018 и 2019 годами. Этот налог будет применяться к российским и иностранным компаниям, ведущим деятельность в России, с исключением нефтяных и нефтеперерабатывающих компаний, угольных компаний, производителей СПГ, малых и средних предприятий (МСП), а также компаний с чистой доналоговой прибылью за 2021-2022 годы менее 1 млрд рублей. Размер налога составляет 10% от суммы превышения прибыли над показателем за 2018-2019 годы, с возможностью досрочной уплаты с 1 октября по 30 ноября 2023 года в размере 5%. Законопроект вступит в силу с 1 января 2024 года. Среди компаний, которые могут заплатить этот налог, наибольший разовый сбор ожидается у Сбербанка (20,4 млрд рублей), Норникеля (18,3 млрд рублей) и НЛМК (15 млрд рублей), а также у ФосАгро (12,1 млрд рублей), Русала (10,4 млрд рублей), Северстали (9,5 млрд рублей), ММК (9,2 млрд рублей), ВТБ (6,8 млрд рублей), Полюса (6,3 млрд рублей) и ПИК (5,3 млрд рублей)."
DataSet.loc[4, 'summarize'] = "Автор рассказывает о своем опыте инвестирования в российский рынок, упоминая различные пакеты префов, акции и другие финансовые инструменты, включая Сургутнефтегаз, Сбербанк, Норникель, ФосАгро, а также ОФЗ, бонды Уральской стали, гдр Русагро, замещайки Газпрома и GLDRUB_TOM. Он выражает опасения относительно акций ВТБ, критикуя топ-менеджмент банка за обещания дивидендов. В продолжении история рассказывает о трудностях, с которыми сталкиваются инвесторы на российском рынке, включая проблемы с операциями купли/продажи паев ПИФов, которые вынуждают держать большое количество наличных денег, не вложенных в бумаги, что приводит к недостаточному росту ПИФов вслед за рынком. Также упоминаются потери на заморозке акций и фондов на США, которые стали большим финансовым уроком для инвесторов."
DataSet.loc[5, 'summarize'] = "В тексте обсуждаются текущие экономические события и финансовые новости, включая изменения в курсе доллара и акций различных компаний. После скачков в феврале, доллар мог быть приобретен практически бесплатно за 50-55 рублей, что, по мнению автора, указывает на искусственное занижение курса валюты. Автор предполагает, что рост курса доллара может продолжаться, и курс может даже опуститься ниже текущего уровня. Рост импорта и выход недружественных нерезидентов с рынка также окажут поддержку доллару. В тексте упоминаются первые словесные интервенции, связанные с курсом рубля, и предполагаемые санкции против Лисина и НЛМК, которые могут быть включены в 11 пакет санкций Евросоюза. Также обсуждаются акции компаний, таких как OZON, FIVE, X5 Group, Татнефть, Сегежа, Whoosh, и Лукойл, а также планы компаний на развитие и изменение юрисдикции."
DataSet.loc[6, 'summarize'] = "Итоги недели с 27.03.23 по 01.04.23 включают обзоры по крепкому газовику Новатэк и Globaltrans, а также отчеты от компаний Сегежа и ЦИАН. В тексте также упоминаются важные события, такие как планы китайского гиганта Aliababa на проведение IPO для шести сегментов своего бизнеса, попытки спекулянтов подбить обзором банк Metropolitan Bank Holding Corp., а также заявления ФРС США о влиянии повышения процентных ставок на экономику и инфляцию. Министерство энергетики России указывает на неточность иностранных референсов по ценам на нефть из России и начинает работу по формированию отечественных индикаторов. Также планируется проведение вебинара на тему перспективы 2023 года для рынков РФ и США."
DataSet.loc[7, 'summarize'] = "Выручка VK в 1 квартале 2023 года составила 25,9 млрд рублей, что на 16,2% меньше, чем в предыдущем квартале. Это свидетельствует о консенсус-прогнозе, составленном Интерфаксом на основе опроса аналитиков инвесткомпаний и банков. В целом, выручка VK в 2023 году составила около 133 млрд рублей, что на 35% больше, чем в предыдущем году, и является наивысшим показателем за наблюдаемый период 1."
DataSet.loc[8, 'summarize'] = "Индекс МосБиржи в среду прибавил 0,8%, несмотря на снижение котировок ВТБ на 6,3% после сильного роста в предыдущие сессии. Банк опубликовал отчетность за 2022 год, в которой сообщил о непланируемой выплате дивидендов в ближайшем будущем. В лидерах роста оказались акции РусГидро, а также акции КАМАЗа, Соллерса и расписки Глобалтрак. Владелец Глобалтрака ведет переговоры о продаже бизнеса, потенциальным покупателем может стать группа «Монополия». Акции Группы ГАЗ также показали рост после новостей об иске к Volkswagen на 15 млрд рублей."
DataSet.loc[9, 'summarize'] = "Bashneft планирует выплатить дивиденды за 2022 год в размере 35.5 миллиардов рублей по ставке 199.89 рублей за акцию, что включает 29.55 миллиардов рублей на обыкновенных акциях и 5.95 миллиардов рублей на предпочтительных акциях типа A 6."
DataSet.loc[10, 'summarize'] = "#MRKU = +5% = макс за 7 мес"
DataSet.loc[11, 'summarize'] = "В 2022 году «Яндекс» почти вдвое увеличил продажи потребительской электроники и впервые стал лидером в этом сегменте, занимая долю рынка в 13,2% по числу проданных гаджетов. Это стало возможным на фоне ухода иностранных вендоров с российского рынка, что привело к сокращению объема продаж потребительской электроники, компьютерной техники и мобильных устройств на 15,4% до около 2,2 трлн рублей. Все российские марки, способные продолжать импорт электроники или производить ее внутри страны с использованием ввезенных комплектующих, стали бенефициарами 2022 года. Среди них «Яндекс», Hiper, F+, «Гравитон», Nerpa, ICL, большинство из которых связаны с крупными российскими дистрибуторами. В 2023 году рынок потребительской электроники оживился, но, как предполагается, не сможет приблизиться к показателям 2020–2021 годов ."
DataSet.loc[12, 'summarize'] = "Сербия планирует стать первой страной в Европе, где будут использоваться беспилотные автомобили, благодаря сотрудничеству с российским Яндексом. Этот проект включает в себя работу специалистов, которые уже демонстрируют высокий уровень профессионализма в области беспилотных технологий. Президент Сербии Александр Вучич выразил восхищение работой Яндекса, сравнив его с американским Google, подчеркнув, что русский Яндекс играет ключевую роль в развитии технологий в стране."
DataSet.loc[13, 'summarize'] = "суд частично снял арест с активов Volkswagen в России"
DataSet.loc[14, 'summarize'] = "Группа ГАЗ (GAZA) имеет целевую цену в случае полного удовлетворения иска в 1250 рублей, что представляет потенциал роста на 80% от текущей цены акции. Это оценка, предложенная в контексте анализа и прогнозов на 2023 год, где акции Группы ГАЗ рассматриваются как один из потенциально перспективных инвестиционных вариантов "
DataSet.loc[15, 'summarize'] = "На сегодняшний день, 1 мая 2023 года, Банк России с 1 мая повышает макропруденциальные требования по ипотечным кредитам. Эта мера направлена на ограничение рисков заемщиков и банков. В то же время, торги на Московской бирже не проводятся, а на Санкт-Петербургской бирже торги проводятся, но с ограничениями. В экономике ожидаются данные по Manufacturing PMI априля из Японии, Индии, Канады, США, а также по Construction Spending марта и ISM Manufacturing PMI априля из США. Популярные отчетности по потребительским компаниям включают отчеты от Norwegian Cruise Line, Microstrategy, NXPI Semiconductors, и MGM Resorts."
DataSet.loc[16, 'summarize'] = "Мосбиржа назвала самые популярные акции марта 2023 года среди частных инвесторов, которые составляют 31,9% портфеля. Среди них: SBER: 31,9% SBERp: 8,5% GAZP: 20,6% LKOH: 9,5% GMKN: 8% ROSN: 4,9% SNGSp: 4,8% NVTK: 4,2% YNDX: 4,1% CHMF: 3,5% Эти данные отражают предпочтения частных инвесторов в марте 2023 года, подчеркивая важность этих акций в портфелях инвесторов."
DataSet.loc[17, 'summarize'] = "GAZA = +6%"
DataSet.loc[18, 'summarize'] = "GAZA = +15% = макс за 2 года"
DataSet.loc[19, 'summarize'] = "В статье БКС Мир Инвестиций представлены голубые фишки с потенциалом роста более 50% на рынке акций России в 2023 году. Среди них: Яндекс (YNDX): Рекомендация Покупать с целью на год в 3500 рублей, что составляет прирост в 82%. Татнефть (TATN): Рекомендация Покупать с целью на год в 610 рублей, что составляет прирост в 56%. Полюс (PLZL): Рекомендация Покупать с целью на год в 15000 рублей, что составляет прирост в 55%. Эти компании выделяются как лидеры своих отраслей, ведут прозрачную финансовую политику, имеют драйверы для восстановления акций и позитивную динамику бизнес-показателей. Автор статьи предлагает инвесторам рассмотреть эти акции в качестве потенциально перспективных инвестиций"

DataSet.head(5)

,CleanSentences,summarize
0,#SELG #дивиденд сд Селигдар: дивиденды 2022г ...,Селигдар не выплачивает дивиденды за 2022 год.
1,Ozon продолжает развивать специализированные ф...,Ozon запускает новый финтех-продукт «Деньги на...
2,​Фокусы продолжаютсяАкции и инвестиции ВТБ +5...,ВТБ показывает рост акций на 5.1% после падени...
3,​​Windfall Tax — налог на сверхприбыль. Какие ...,В России введена новая форма налога - Windfall...
4,"У нас было 2 пакета префов Сургутнефтегаза, 75...",Автор рассказывает о своем опыте инвестировани...


In [42]:
from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(DataSet, test_size=0.25, random_state=42)

In [50]:
%%time

# Настройка аргументов обучения
training_args = TrainingArguments(
    output_dir='./results',          # директория для сохранения результатов
    num_train_epochs=3,              # количество эпох обучения
    per_device_train_batch_size=16, # размер батча для обучения
    per_device_eval_batch_size=64,   # размер батча для оценки
    warmup_steps=500,                # количество шагов для разогрева
    weight_decay=0.01,               # коэффициент регуляризации
    logging_dir='./logs',            # директория для логов
    logging_steps=10,                # шаги для логирования
)

# Создание тренера
trainer = Trainer(
    model=model,                         # модель
    args=training_args,                 # аргументы обучения
    train_dataset=train_data,         # обучающая выборка
    eval_dataset=valid_data,            # валидационная выборка
    tokenizer=tokenizer,                 # токенизатор
)

# Обучение модели
trainer.train()

# Оценка модели
trainer.evaluate()

# Сохранение модели
model.save_pretrained('./fine_tuned_model')


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`